# Tensorflow intro

This tutorial shows the basic usage of tensorflow to train neural networks

In [19]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import datetime

2023-04-17 14:36:29.925960: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

# Simple NN for classification

## Data processing
Read data and convert them to numerical inputs

In [3]:
X, y = make_classification(n_samples=20000, n_features=8, n_informative=5, 
                           n_redundant=0, n_classes=2, random_state=1)

In [4]:
X, X_test, y, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [5]:
X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [6]:
print('train size: {}, target_ratio: {:.3f}'.format(X_train.shape, np.mean(y_train)))
print('test size: {}, target_ratio: {:.3f}'.format(X_test.shape, np.mean(y_test)))
print('dev size: {}, target_ratio: {:.3f}'.format(X_dev.shape, np.mean(y_dev)))

train size: (12800, 8), target_ratio: 0.502
test size: (4000, 8), target_ratio: 0.501
dev size: (3200, 8), target_ratio: 0.502


## Building a simple model with tf.keras

Very useful documentations with many examples and detailed explanation of everything you might need:
 - https://www.tensorflow.org/api_docs/python/tf/keras/
 - https://keras.io/api/

Contain everything about:
  - Model building: Activations, Losses, Optimizers, Regularization
  - Data processing
  - Pretrained models and datasets
  - Automatic differentiation
  - ...

  

### Model speficication

three APIs for building the model
   - sequential - easy to code, but less flexible - we will use it sometimes
   - functional - flexible and still easy to code - we will use it the most
   - model subclassing - rather complicated and not very much used - we will skip it

#### Sequential API

(https://www.tensorflow.org/guide/keras/sequential_model)

Easy to code but <span style="color:red"> NOT </span> appropriate when:

- Your model has multiple inputs or multiple outputs
- Any of your layers has multiple inputs or multiple outputs
- You need to do layer sharing
- You want non-linear topology (e.g. a residual connection, a multi-branch model)

In [7]:
# Specification A)

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer([X_train.shape[1],]), # Create input layer with 'input data' neurons
    tf.keras.layers.Dense(10, activation="relu"), # Create hidden layer with 10 neurons and ReLU activation
    tf.keras.layers.Dense(1, activation="sigmoid"), # Create output layer with one neuron and sigmoid activation
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                90        
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Specification B)

model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(X_train.shape[1],)))
model.add(tf.keras.layers.Dense(10, activation="relu"))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 10)                90        
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________


#### Functional API

(https://www.tensorflow.org/guide/keras/functional)

The Keras functional API is a way to create models that are more flexible than the tf.keras.Sequential API. The functional API can handle models with non-linear topology, shared layers, and even multiple inputs or outputs.

The main idea is that a deep learning model is usually a directed acyclic graph (DAG) of layers. So the functional API is a way to build graphs of layers.



In [9]:
inputs = tf.keras.Input(shape=(X_train.shape[1],))

hidden = tf.keras.layers.Dense(10)(inputs)
hidden = tf.keras.activations.relu(hidden)
hidden = tf.keras.layers.Dense(1)(hidden)
outputs = tf.keras.activations.sigmoid(hidden)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name='Model')

In [10]:
model.summary()

Model: "Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 8)]               0         
                                                                 
 dense_4 (Dense)             (None, 10)                90        
                                                                 
 tf.nn.relu (TFOpLambda)     (None, 10)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
 tf.math.sigmoid (TFOpLambda  (None, 1)                0         
 )                                                               
                                                                 
Total params: 101
Trainable params: 101
Non-trainable params: 0
_______________________________________________________________

In [11]:
tf.keras.utils.plot_model(model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


### Model compilation and training

In [12]:
# compile the model with selected optimizer, loss and metrics
model.compile(
        optimizer=tf.optimizers.Adam(), # Several other possibilities for optimizers 
        loss=tf.losses.BinaryCrossentropy(), # Select the proper loss for the task
        metrics=[tf.keras.metrics.AUC(), tf.keras.metrics.BinaryAccuracy()], # Select the proper metrics for the task
)

In [13]:
print('\n>>> Bias of the last layers:')
print(model.layers[3].weights[1].numpy())

print('\n>>> Kernel of the last layers:')
print(model.layers[3].weights[0].numpy())

print('\n>>> Bias of the first layers:')
print(model.layers[1].weights[1].numpy())

print('\n>>> Kernel of the first layers:')
print(model.layers[1].weights[0].numpy())


>>> Bias of the last layers:
[0.]

>>> Kernel of the last layers:
[[-0.56460613]
 [ 0.29604632]
 [ 0.21662462]
 [-0.6775259 ]
 [ 0.30738193]
 [-0.31533614]
 [-0.17806965]
 [-0.2903523 ]
 [ 0.17737854]
 [ 0.5742884 ]]

>>> Bias of the first layers:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

>>> Kernel of the first layers:
[[-1.9635808e-01  4.0544736e-01 -3.5376328e-01  1.5955198e-01
  -5.0685287e-02 -7.6352417e-02  2.4730825e-01  3.3675057e-01
  -4.0275934e-01  3.2808918e-01]
 [ 2.3958725e-01  2.8455967e-01 -5.0527781e-01 -2.6963496e-01
  -7.9658508e-02  4.7420633e-01  5.5414319e-02  2.0938939e-01
   3.5862774e-01 -5.7387954e-01]
 [ 5.1212227e-01 -3.6785352e-01  4.4393539e-04  8.4377170e-02
   2.7562928e-01  1.5110874e-01 -3.5856718e-01  4.6589088e-01
   3.3835709e-02  5.6500971e-01]
 [-2.1937844e-01  3.5303003e-01  3.8683599e-01 -1.6998386e-01
   2.5893134e-01  2.9614133e-01  3.0482227e-01  4.0505934e-01
  -2.8401744e-01  2.7615428e-01]
 [-3.5402387e-01 -3.9353395e-01  2.9713529e-01 -1.6215485e

In [14]:
# train the model with default setting
model.fit(X_train, y_train, batch_size=64, epochs=100)

Epoch 1/100
200/200 [==============================] - 1s 3ms/step - loss: 0.6721 - auc: 0.6333 - binary_accuracy: 0.5921
Epoch 2/100
200/200 [==============================] - 0s 2ms/step - loss: 0.5180 - auc: 0.8405 - binary_accuracy: 0.7606
Epoch 3/100
200/200 [==============================] - 1s 2ms/step - loss: 0.4370 - auc: 0.8913 - binary_accuracy: 0.8145
Epoch 4/100
200/200 [==============================] - 1s 3ms/step - loss: 0.3860 - auc: 0.9148 - binary_accuracy: 0.8426
Epoch 5/100
200/200 [==============================] - 1s 3ms/step - loss: 0.3548 - auc: 0.9276 - binary_accuracy: 0.8594
Epoch 6/100
200/200 [==============================] - 1s 3ms/step - loss: 0.3352 - auc: 0.9348 - binary_accuracy: 0.8683
Epoch 7/100
200/200 [==============================] - 1s 5ms/step - loss: 0.3231 - auc: 0.9386 - binary_accuracy: 0.8743
Epoch 8/100
200/200 [==============================] - 1s 3ms/step - loss: 0.3143 - auc: 0.9414 - binary_accuracy: 0.8773
Epoch 9/100
200/200 [===

In [15]:
# Evaluate the model and predict for the test data
model.evaluate(X_test, y_test)
test_pred = model.predict(X_test)

125/125 [==============================] - 0s 2ms/step


In [16]:
for pred, true in zip(test_pred, y_test[0:10]):
    print('{} - {:.2f}'.format(true, pred[0]))

1 - 0.33
0 - 0.19
0 - 0.55
1 - 0.99
0 - 0.00
0 - 0.00
1 - 0.98
0 - 0.00
1 - 1.00
1 - 1.00


### Add early stopping and regularization

In [17]:
# Input layer
inputs = tf.keras.Input(shape=(X_train.shape[1]))

# Hidden layer with regularization and ReLU
hidden = tf.keras.layers.Dense(10, kernel_regularizer=tf.keras.regularizers.l2(0.001))(inputs)
hidden = tf.keras.activations.relu(hidden)

# Output layer with regularization and sigmoid
outputs = tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.l2(0.001))(hidden)
outputs = tf.keras.activations.sigmoid(outputs)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name='RegularizedModel')

model.compile(
        optimizer=tf.optimizers.Adam(),
        loss=tf.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.AUC(), tf.keras.metrics.BinaryAccuracy()],
)

model.summary()

Model: "RegularizedModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 8)]               0         
                                                                 
 dense_6 (Dense)             (None, 10)                90        
                                                                 
 tf.nn.relu_1 (TFOpLambda)   (None, 10)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 11        
                                                                 
 tf.math.sigmoid_1 (TFOpLamb  (None, 1)                0         
 da)                                                             
                                                                 
Total params: 101
Trainable params: 101
Non-trainable params: 0
____________________________________________________

In [18]:
epochs = 200

early_call = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)

model.fit(X_train,
          y_train,
          epochs=epochs,
          validation_data=(X_dev, y_dev),
          callbacks=[early_call])

Epoch 1/200
400/400 [==============================] - 2s 3ms/step - loss: 0.5258 - auc_1: 0.8369 - binary_accuracy: 0.7527 - val_loss: 0.4314 - val_auc_1: 0.9059 - val_binary_accuracy: 0.8325
Epoch 2/200
400/400 [==============================] - 1s 2ms/step - loss: 0.4007 - auc_1: 0.9154 - binary_accuracy: 0.8416 - val_loss: 0.3614 - val_auc_1: 0.9344 - val_binary_accuracy: 0.8681
Epoch 3/200
400/400 [==============================] - 1s 2ms/step - loss: 0.3503 - auc_1: 0.9365 - binary_accuracy: 0.8709 - val_loss: 0.3292 - val_auc_1: 0.9452 - val_binary_accuracy: 0.8828
Epoch 4/200
400/400 [==============================] - 1s 2ms/step - loss: 0.3259 - auc_1: 0.9449 - binary_accuracy: 0.8822 - val_loss: 0.3143 - val_auc_1: 0.9494 - val_binary_accuracy: 0.8888
Epoch 5/200
400/400 [==============================] - 1s 2ms/step - loss: 0.3130 - auc_1: 0.9490 - binary_accuracy: 0.8895 - val_loss: 0.3052 - val_auc_1: 0.9521 - val_binary_accuracy: 0.8925
Epoch 6/200
400/400 [==============

### Gridsearch and Tensorboard
Run gridsearch over hidden layer size, L2 regularization, activation, check the outputs in Tensorboard

I recommend not to run Tensorboard from Jupyter notebook but from terminal directly

use "tensorboard --logdir logs" in command line

In [20]:
# hidden_sizes = [2, 5, 10, 20, 50]
# l2_regs = [0.01, 0.001, 0.0001]
# activations = ['relu', 'tanh']

hidden_sizes = [2, 5]
l2_regs = [0.01]
activations = ['relu', 'tanh']

epochs = 10
batch_size = 64

early_call = tf.keras.callbacks.EarlyStopping(monitor='val_AUC', mode='max', patience=10, restore_best_weights=True)

for activation in activations:
    for l2_reg in l2_regs:
        for hidden_size in hidden_sizes:
            if activation == 'relu':
                activate = tf.keras.activations.relu
            elif activation == 'tanh':
                activate = tf.keras.activations.tanh

            # Create Tensorboard Callback
            param_string = 'act-{},l2-{},hs-{}'.format(activation, l2_reg, hidden_size)
            log_dir = 'logs/binary_classification_test/' + param_string
            tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

            # Input layer
            inputs = tf.keras.Input(shape=(X_train.shape[1]))

            # Hidden layer with regularization and ReLU
            hidden = tf.keras.layers.Dense(hidden_size, kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(inputs)
            hidden = activate(hidden)

            # Output layer with regularization and sigmoid
            outputs = tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(hidden)
            outputs = tf.keras.activations.sigmoid(outputs)

            model = tf.keras.Model(inputs=inputs, outputs=outputs, name='RegularizedModel')

            model.compile(
                    optimizer=tf.optimizers.Adam(),
                    loss=tf.losses.BinaryCrossentropy(),
                    metrics=[tf.keras.metrics.AUC(name='AUC'), tf.keras.metrics.BinaryAccuracy()],
            )

            # Train the model
            model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,
                      validation_data=(X_dev, y_dev),
                      callbacks=[early_call, tensorboard_callback])

Epoch 1/10
200/200 [==============================] - 1s 4ms/step - loss: 1.4222 - AUC: 0.3181 - binary_accuracy: 0.4266 - val_loss: 1.1172 - val_AUC: 0.3441 - val_binary_accuracy: 0.4150
Epoch 2/10
200/200 [==============================] - 1s 3ms/step - loss: 0.9371 - AUC: 0.3946 - binary_accuracy: 0.4161 - val_loss: 0.8267 - val_AUC: 0.4370 - val_binary_accuracy: 0.4284
Epoch 3/10
200/200 [==============================] - 1s 3ms/step - loss: 0.7616 - AUC: 0.4931 - binary_accuracy: 0.4731 - val_loss: 0.7232 - val_AUC: 0.5255 - val_binary_accuracy: 0.5056
Epoch 4/10
200/200 [==============================] - 1s 3ms/step - loss: 0.6966 - AUC: 0.5700 - binary_accuracy: 0.5573 - val_loss: 0.6790 - val_AUC: 0.5892 - val_binary_accuracy: 0.5884
Epoch 5/10
200/200 [==============================] - 1s 3ms/step - loss: 0.6667 - AUC: 0.6232 - binary_accuracy: 0.6128 - val_loss: 0.6554 - val_AUC: 0.6394 - val_binary_accuracy: 0.6281
Epoch 6/10
200/200 [==============================] - 1s 3ms